# Center Construction for Tambara Yamagami Categories.
The code below does the following:
1. creates a list of all abelian groups of order within a range prescribed by the user
2. creates all bilinear forms for each abelian group up to isomorphism
3. for each group and bilinear form, creates the two Tambara Yamagami categories that correspond
4. for each TB category, creates the S and T matrices of the center construction.

In [1]:
from sage.metric_groups.metric_groups import *
import itertools, pandas as pd

In [2]:
G = AdditiveAbelianGroup([3])
Q = matrix([[8]])
Z = CyclotomicField(12)
GQ = PreMetricGroup_class(G, Q, Z)

In [3]:
GQ

Additive abelian group isomorphic to Z/3 with quadratic form determined by 
[8]
whose codomain is the Cyclotomic Field of order 12 and degree 4

In [4]:
GQ = MetricGroup([(3,1,1),(5,1,1),(2,3,6)]); GQ

Additive abelian group isomorphic to Z/3 + Z/5 + Z/8 + Z/8 with quadratic form determined by 
[160   0   0   0]
[  0 144   0   0]
[  0   0  30  15]
[  0   0  15  30]
whose codomain is the Cyclotomic Field of order 240 and degree 64

In [6]:
?G
g = GQ.0

In [13]:
g.bilinear(g)

zeta240^40 - 1

In [24]:
df = pd.read_pickle('bil_forms.pkl')

In [25]:
start = 2
end = 10000
Bilinear_Forms = df.loc[(df['group_order'] >= start) & (df['group_order'] <= end)]
Bilinear_Forms

,name,group_order,prd_tuples
0,A_2^1,2,"[(2, 1, 1)]"
1,A_3^1,3,"[(3, 1, 1)]"
2,B_3^1,3,"[(3, 1, 2)]"
3,A_2^2,4,"[(2, 2, 1)]"
4,B_2^2,4,"[(2, 2, 2)]"
...,...,...,...
162545,E_2^1+E_2^1+A_5^2+A_5^1+A_5^1,10000,"[(2, 1, 5), (2, 1, 5), (5, 2, 1), (5, 1, 1), (..."
162546,A_2^1+A_2^1+A_2^1+A_2^1+B_5^2+A_5^1+A_5^1,10000,"[(2, 1, 1), (2, 1, 1), (2, 1, 1), (2, 1, 1), (..."
162547,E_2^1+E_2^1+B_5^2+A_5^1+A_5^1,10000,"[(2, 1, 5), (2, 1, 5), (5, 2, 2), (5, 1, 1), (..."
162548,A_2^1+A_2^1+A_2^1+A_2^1+A_5^1+A_5^1+A_5^1+A_5^1,10000,"[(2, 1, 1), (2, 1, 1), (2, 1, 1), (2, 1, 1), (..."


In [26]:
def cyclotomic_n(pr_orders):
    n = prod(pr_orders)
    n1 = lcm(pr_orders)
    #n1 corresponds to denominator of the quadratic form
    if n1 % 2 == 0:
        n1 = n1*2
    #if n1 is odd, gauss sums will exist in QQ(i)
    else:
        n1 = n1*4
    #We need a cyclotomic field that contains sqrt(n)
    n2 = n.squarefree_part()
    if n2 % 2 == 0:
        n2 = n2 * 4
    n3 = lcm(n1,n2)
    return n3

In [39]:
def build_metric_group(prd_tuples):
    if prd_tuples in ZZ:
        prd_tuples = list(Bilinear_Forms.prd_tuples)[prd_tuples]
    orders = [p^r for (p,r,d) in prd_tuples for i in range(int(d/5)+1)]
    n = cyclotomic_n(orders)
    G = AdditiveAbelianGroup(orders)
    Z = CyclotomicField(n)
    z = Z.gen()
    Q = []
    for tup in prd_tuples:
        p,r,d = tup
        if p == 2 and d == 1:
            Q.append(matrix([[n*p^(-r-1)]]))
        elif p == 2 and d == 2:
            Q.append(matrix([[-n*p^(-r-1)]]))
        elif p == 2 and d == 3:
            Q.append(matrix([[5*n*p^(-r-1)]]))
        elif p == 2 and d == 4:
            Q.append(matrix([[-5*n*p^(-r-1)]]))
        elif p == 2 and d == 5:
            Q.append(n*p^(-r-1)*matrix([[0,1],[1,0]]))
        elif p == 2 and d == 6:
            Q.append(n*p^(-r-1)*matrix([[2,1],[1,2]]))
        elif d == 1:
            v = (p^r + 1)/2 #v is 2^(-1) in Z/p^rZ
            Q.append(matrix([[ZZ(v*n*p^(-r))]]))
        elif d == 2:
            v = (p^r + 1)/2 #v is 2^(-1) in Z/p^rZ
            u = ZZ(Zmod(p).multiplicative_generator())
            Q.append(matrix([[ZZ(u*v*n*p^(-r))]]))
        else:
            print("Build error: ", p, r, d)
    return [G,block_diagonal_matrix(Q),z]

The functions `q` and `b` are multiplicative and the functions `q2` and `b2` are additive.

In [40]:
def q(g,Q,z):
    x = vector(g.lift())
    return z^(x*Q*x)

def b(g,h,Q,z):
    return q(g+h,Q,z)/q(g,Q,z)/q(h,Q,z)

def q_a(g,Q,a,z):
    return q(g,Q,z)*b(g,a,Q,z)

def q2(g,Q,z):
    n = z.multiplicative_order()
    x = vector(g.lift())
    return 1/n*(x*Q*x % n)

def b2(g,h,Q,z):
    return q2(g+h,Q,z)-q2(g,Q,z)-q2(h,Q,z)

In [41]:
def sqrt_(n,z):
    n3 = z.multiplicative_order()
    sqrt_n = 1
    for fac in factor(n.squarefree_part()):
        if fac[0] == 2:
            sqrt_n = z^(n3/8) + z^(7*n3/8)
        else:
            p = fac[0]
            sqrt_n = sqrt_n * (1-z^(n3/4))/(1-z^(n3/4*legendre_symbol(-1,p))) * sum([legendre_symbol(i,p)*z^(n3*i/p) for i in range(p)])
    return sqrt_n * sqrt(n/n.squarefree_part())

In [42]:
G, Q, z = build_metric_group([(3,1,1)])
sgn_tau = 1

In [43]:
n = G.order()
sqrt_n = sqrt_(n,z)
tau = sgn_tau * sqrt_n/n
print("tau", tau, tau^2)

gauss = sqrt_n/n*sum([q(g,Q,z) for g in G])
print("gauss", gauss)

#Checks if we can find the Delta's by computing square roots in QQ(z_n)
if not product([(sgn_tau*gauss/q(a,Q,z)).is_nth_power(2) for a in G]):
    n1 = z.multiplicative_order()
    Z1 = CyclotomicField(2*n1)
    z = Z1.zeta()
    tau = Z1(tau)
    gauss = Z1(gauss)
    sqrt_n = Z1(sqrt_n)
    Q = 2*Q
    print("new cyclotomic field: ",G,Q,z)
Delta = [(sgn_tau*gauss/q(a,Q,z)).sqrt() for a in G]
pows = [z^(i) for i in range(z.multiplicative_order())]

Delta_pows = [pows.index(d)/z.multiplicative_order() for d in Delta]
print("Delta: ", Delta_pows)

tau -1/3*zeta12^3 + 2/3*zeta12 1/3
gauss -zeta12^3
new cyclotomic field:  Additive abelian group isomorphic to Z/3 [16] zeta24
Delta:  [3/8, 1/24, 1/24]


In [65]:
TB_list = G.list() + [-1]
TB_n = G.order() + 1
#simple objects can be 

The following function computes $N_{X,Y}^Z$, the multiplicity of the simple object $Z$ in $X \otimes Y$.  We have given it the same name as Matthew Titsworth's Mathematica package [here](https://mktitsworth.com/fusion-categories/).  We will continue this practice througout this notebook when we can.

Each of X, Y, and Z are indices of the list `TB_simple`.

In [196]:
def nFunction(X,Y,Z):
    x,y,z = [TB_list[i] for i in [X,Y,Z]]
    if x in G and y in G and z in G:
        return ZZ(x+y==z)
    elif x in G and y in G: #thus z is the m object
        return 0
    elif x in G and z in G: #thus y is the m object
        return 0
    elif y in G and z in G: #thus x is the m object
        return 0
    elif x in G:            #thus y & z are m
        return 1
    elif y in G:            #thus x & z are m
        return 1
    elif z in G:            #thus x & y are m
        return 1
    else:                   #thus x, y, & z are m
        return 0

In [197]:
def nMatrix(X):
    return matrix([[nFunction(X,Y,Z) for Y in range(TB_n)] for Z in range(TB_n)])

In [199]:
nMatrix(3)

[0 0 0 1]
[0 0 0 1]
[0 0 0 1]
[1 1 1 0]

In [ ]:
def Phi_simple(x,y,z):
    #(xy)z to x(yz)
    Phi = []

In [194]:
TB = CombinatorialFreeModule(ZZ, [str(g) for g in G.list()] + ['m'], prefix = 'TB'); TB
b = TB.basis(); b
a = b['(1)'] + 2*b['(2)']
c = b['(2)'] + b['m']

(2)

[1 0 0]
[0 0 1]
[0 1 0]

In [109]:
#import sage
from sage.matrix.matrix import is_Matrix
from sage.groups.group import is_Group
from sage.combinat.free_module import CombinatorialFreeModule
class TBcat(CombinatorialFreeModule):
    def __init__(self, group, quad_form, zeta, sgn_tau):
        assert is_Group(group)
        assert is_Matrix(quad_form)
        assert sgn_tau == 1 or sgn_tau == -1
        self._R = ZZ
        self._basis_keys = [str(g) for g in group.list()] + ['m']
        self._prefix = 'TB'
        self.group = group
        self.quad_form = quad_form
        self.zeta = zeta
        self.sgn_tau = sgn_tau
    def __repr__(self):
        return f"The Grothendieck Ring of a Tambara Yamagami Category with G the {str(group)}, quadratic form defined by {str(1/zeta.multiplicative_order()*quad_form)} and tau sign equal to {sgn_tau}"
    def _mult_(self, right):
        assert self in TB
        assert right in TB
        a = self.to_vector()
        a_G = a[:-1]
        a_m = a[-1]
        b = right.to_vector()
        b_G = b[:-1]
        b_m = b[-1]
        if self and right in G:
            return 

TypeError: unsupported operand parent(s) for *: 'Free module generated by {'(0)', '(1)', '(2)', 'm'} over Integer Ring' and 'Free module generated by {'(0)', '(1)', '(2)', 'm'} over Integer Ring'

In [157]:
from sage.combinat.free_module import CombinatorialFreeModule

In [158]:
G.list()

[(0), (1), (2)]

In [181]:

G_list = G.list()
Gpairs = [(G_list[i],G_list[j]) for i in range(n) for j in range(i)]
Theta_X = [b(g,-g,Q,z) for g in G_list for i in range(2)]
Theta_Y = [b(g,-h,Q,z) for (g,h) in Gpairs]
Theta_Z = [sgn_tau*(-1)^i*d for d in Delta for i in range(2)]
Theta = Theta_X + Theta_Y + Theta_Z
T = block_diagonal_matrix([diagonal_matrix(Theta_X), diagonal_matrix(Theta_Y), diagonal_matrix(Theta_Z)])
show(T)

[                 1                  0                  0                  0                  0                  0|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  1                  0                  0                  0                  0|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0          -zeta24^4                  0                  0                  0|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0                  0          -zeta24^4                  0                  0|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0                  0                  0          -zeta24^4                  0|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0                  0                  0                  0          -zeta24^4|                 0                  0                  0|                 0                  0                  0                  0                  0                  0]
[-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------]
[                 0                  0                  0                  0                  0                  0|                 1                  0                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0                  0                  0                  0                  0|                 0                  1                  0|                 0                  0                  0                  0                  0                  0]
[                 0                  0                  0                  0                  0                  0|                 0                  0       zeta24^4 - 1|                 0                  0                  0                  0                  0                  0]
[-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------]
[                 0                  0                  0                  0                  0                  0|                 0                  0                  0| zeta24^5 - zeta24                  0                  0                  0                  0                  0]
[                 0                  0                  0                  0                  0                  0|                 0                  0                  0|                 0 -zeta24^5 + zeta24                  0                  0                  0                  0]
[                 0                  0                  0                  0                  0                  0|                 0                  0                  0|                 0                  0             zeta24                  0                  0   

In [182]:
SXX = matrix([[b(g,h,Q,z)^(2) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#show(SXX)
SXY = matrix([[2*b(h,a+c,Q,z)^(1) for (a,c) in Gpairs] for h in G_list for i in range(2)])
#show(SXY)
SXZ = matrix([[(-1)^(j)*sqrt_n*q(h,Q,z)^(-1)*q(h,Q,z)^(1)*b(h,g,Q,z)^(1) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#GNN say SXZ = matrix([[(-1)^(j)*sqrt_n*rho(h,Q)^(-1)*rho_a(h,Q,g)^(-1) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#show(SXZ)
SYY = matrix([[2*(b(a,d,Q,z)*b(x,c,Q,z)+b(a,c,Q,z)*b(x,d,Q,z)) for (a,x) in Gpairs] for (c,d) in Gpairs])
#show(SYY)
SYZ = zero_matrix(parent(z), n*(n-1)/2, 2*n)
#show(SYZ)
SZZ = matrix([[Delta[i]^(-1)*(-1)^j*Delta[k]^(-1)*(-1)^l*sum([b(x,x,Q,z)^2*q_a(x,Q,G_list[i],z)^(-1)*q_a(x,Q,G_list[k],z)^(-1) for x in G_list]) for i in range(n) for j in range(2)] for k in range(n) for l in range(2)])
#show(SZZ)
S = block_matrix([[SXX, SXY, SXZ],[SXY.transpose(), SYY, SYZ], [SXZ.transpose(), SYZ.transpose(), SZZ]])/(2*n)
show(S)
dims = [1]*(2*n) + [2]*ZZ(n*(n-1)/2) + [sqrt_n]*(2*n)
p_plus = sum([Theta[i]*dims[i]^2 for i in range(n*(7+n)/2)])
p_minus = sum([Theta[i]^(-1)*dims[i]^2 for i in range(n*(7+n)/2)])

[                         1/6                          1/6                          1/6                          1/6                          1/6                          1/6                          1/3                          1/3                          1/3 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2]
[                         1/6                          1/6                          1/6                          1/6                          1/6                          1/6                          1/3                          1/3                          1/3  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2]
[                         1/6                          1/6                -1/6*zeta24^4                -1/6*zeta24^4           1/6*zeta24^4 - 1/6           1/6*zeta24^4 - 1/6           1/3*zeta24^4 - 1/3                -1/3*zeta24^4                          1/3 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2  1/3*zeta24^6 - 1/6*zeta24^2  1/3*zeta24^6 - 1/6*zeta24^2 -1/6*zeta24^6 - 1/6*zeta24^2 -1/6*zeta24^6 - 1/6*zeta24^2]
[                         1/6                          1/6                -1/6*zeta24^4                -1/6*zeta24^4           1/6*zeta24^4 - 1/6           1/6*zeta24^4 - 1/6           1/3*zeta24^4 - 1/3                -1/3*zeta24^4                          1/3  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2 -1/3*zeta24^6 + 1/6*zeta24^2 -1/3*zeta24^6 + 1/6*zeta24^2  1/6*zeta24^6 + 1/6*zeta24^2  1/6*zeta24^6 + 1/6*zeta24^2]
[                         1/6                          1/6           1/6*zeta24^4 - 1/6           1/6*zeta24^4 - 1/6                -1/6*zeta24^4                -1/6*zeta24^4                -1/3*zeta24^4           1/3*zeta24^4 - 1/3                          1/3 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 + 1/3*zeta24^2 -1/6*zeta24^6 - 1/6*zeta24^2 -1/6*zeta24^6 - 1/6*zeta24^2  1/3*zeta24^6 - 1/6*zeta24^2  1/3*zeta24^6 - 1/6*zeta24^2]
[                         1/6                          1/6           1/6*zeta24^4 - 1/6           1/6*zeta24^4 - 1/6                -1/6*zeta24^4                -1/6*zeta24^4                -1/3*zeta24^4           1/3*zeta24^4 - 1/3                          1/3  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2  1/6*zeta24^6 + 1/6*zeta24^2  1/6*zeta24^6 + 1/6*zeta24^2 -1/3*zeta24^6 + 1/6*zeta24^2 -1/3*zeta24^6 + 1/6*zeta24^2]
[                         1/3                          1/3           1/3*zeta24^4 - 1/3           1/3*zeta24^4 - 1/3                -1/3*zeta24^4                -1/3*zeta24^4                 1/3*zeta24^4          -1/3*zeta24^4 + 1/3                         -1/3                            0                            0                            0                            0                            0                            0]
[                         1/3                          1/3                -1/3*zeta24^4                -1/3*zeta24^4           1/3*zeta24^4 - 1/3           1/3*zeta24^4 - 1/3          -1/3*zeta24^4 + 1/3                 1/3*zeta24^4                         -1/3                            0                            0                            0                            0                            0                            0]
[                         1/3                          1/3                          1/3                          1/3                          1/3                          1/3                         -1/3                         -1/3                         -1/3                            0                            0                            0                            0                            0                            0]
[-1/6*zeta24^6 + 1/3*zeta24^2  1/6*zeta24^6 - 1/3*zeta24^2 -1/6*zet

In [183]:
C = S*S
show(C)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

In [184]:
show((S*T^(-1))^3)

[   1    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    1    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    1    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    1    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    1]
[   0    0    0    0    0    0    0    0    0    0    0    1    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0]

In [59]:
TB_data = [T + [(-1)^i] for T in full_met_groups for i in range(2)]
tup = TB_data[21]
G = tup[0]
G

Additive abelian group isomorphic to Z/2 + Z/3

In [41]:
def rho(g,Q1):
    return z^(vector(g.lift())*Q1*(vector(g.lift())))

def rho_a(g,Q1,h):
    return z^(vector(g.lift())*Q1*(vector(g.lift()) + 2*vector(h.lift())))

def chi(g,h, Q1):
    return z^(vector(g.lift())*2*Q1*vector(h.lift()))

In [62]:
ind = 42
#Pick one of the TB Category input datas
show(TB_data[ind])
G, met, Q, z, sgn_tau = TB_data[ind]
n = G.order()
n3 = z.multiplicative_order()
sqrt_n = 1
for fac in factor(n.squarefree_part()):
    if fac[0] == 2:
        sqrt_n = z^(n3/8) + z^(7*n3/8)
    else:
        p = fac[0]
        sqrt_n = sqrt_n * (1-z^(n3/4))/(1-z^(n3/4*legendre_symbol(-1,p))) * sum([legendre_symbol(i,p)*z^(n3*i/p) for i in range(p)])
sqrt_n = sqrt_n * sqrt(n/n.squarefree_part())
tau = sgn_tau * sqrt_n/n
#GNN gauss
gauss = sqrt_n*sum([rho(g, Q) for g in G])/n
#RTJ gauss
#gauss = sqrt_n*sum([rho(g, Q)^(-1) for g in G])/n
show(gauss)

[
                                                                [8 0]
Additive abelian group isomorphic to Z/3 + Z/3, [(3, 1, 2, 1)], [0 8],

zeta12, 1
]

-1

In [64]:
H = G.submodule([2*g for g in G])
quotient_gens = [g for g in G.gens() if g not in H]
coset_reps = [G.zero()] + [sum(K) for K in Set(quotient_gens).subsets()][1:]
#List elements of G such that they are grouped by cosets of {2g for g in G}
G_list = [c + h for c in coset_reps for h in H]
G_halves = zip(*[[g,2*g] for g in G_list])
c_n = len(coset_reps)
#Building the Deltas with correct +/- is tricky.  First, define them for the coset reps of G / {2g for g in G}
#GNN Deltas
Delta_for_cos_reps = [(sgn_tau*q(c,Q,z)^(-1)*gauss).nth_root(2) for c in coset_reps]
#RTJ Deltas
#Delta_for_cos_reps = [(sgn_tau*rho(c, Q)*gauss).nth_root(2)^(-1) for c in coset_reps]
#Then Delta_{2a+c} = Delta_c * chi(-a,a+c) where 2a in H.
Delta = []
for i in range(c_n):
    D = Delta_for_cos_reps[i]
    c = coset_reps[i]
    for h in H:
        #Need to divide h by 2, but Sage does not allow this in Z-modules.
        a = G_halves[0][G_halves[1].index(h)]
        Delta.append(D*b(-a,a+c,Q,z))
show(Delta)
show(coset_reps)
show(H.list())
show(G_list)

ValueError: -5/3 not a 2-th root in Cyclotomic Field of order 20 and degree 8

In [347]:
for c in G_list:
    print c, G_list[floor(G_list.index(c) / H.order())*H.order()]
    #print G_list[G_list.index(c) % H.order()]

(0) (0)
(1) (0)
(2) (0)


In [473]:
#This should be fixed to deal with this on a group-theory level, but I cannot understand it at this moment.
def XZ_negation_fix(a,b, Q, G_list, Delta):
    j = G_list.index(b)
    k = G_list.index(2*a+b)
    return chi(-a,b+a,Q)*Delta[k]/Delta[j]

show(H.list())
show(G_list)
correction = []
for a in G_list:
    correction.append([])
    for b in G_list:
        correction[-1:][0].append(XZ_negation_fix(a,b, Q, G_list, Delta))
show(matrix(correction))

[(0), (1), (2)]

[(0), (1), (2)]

[1 1 1]
[1 1 1]
[1 1 1]

In [442]:
pows = [z^(i) for i in range(n3)]
#Delta = [(tau*sum([rho_a(g,Q,h) for g in G])).nth_root(2)^(-1) for h in G]
Gpairs = [(G_list[i],G_list[j]) for i in range(n) for j in range(i)]
Theta_X = [chi(g,-g,Q) for g in G_list for i in range(2)]
Theta_Y = [chi(g,-h,Q) for (g,h) in Gpairs]
Theta_Z = [sgn_tau*(-1)^i*d for d in Delta for i in range(2)]
Theta = Theta_X + Theta_Y + Theta_Z
#show(Theta_X)
#show(Theta_Y)
#show(Theta_Z)
T = block_diagonal_matrix([diagonal_matrix(Theta_X), diagonal_matrix(Theta_Y), diagonal_matrix(Theta_Z)])
#show(T)
SXX = matrix([[chi(g,h,Q)^(2) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#show(SXX)
SXY = matrix([[2*chi(h,a+b, Q)^(1) for (a,b) in Gpairs] for h in G_list for i in range(2)])
#show(SXY)
SXZ = matrix([[(-1)^(j)*sqrt_n*rho(h,Q)^(-1)*rho_a(h,Q,g)^(1)*XZ_negation_fix(h,g,Q,G_list,Delta) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#GNN say SXZ = matrix([[(-1)^(j)*sqrt_n*rho(h,Q)^(-1)*rho_a(h,Q,g)^(-1) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
show(SXZ)
SYY = matrix([[2*(chi(a,d,Q)*chi(b,c,Q)+chi(a,c,Q)*chi(b,d,Q)) for (a,b) in Gpairs] for (c,d) in Gpairs])
#show(SYY)
SYZ = zero_matrix(parent(z), n*(n-1)/2, 2*n)
#show(SYZ)
SZZ = matrix([[Delta[i]^(-1)*(-1)^j*Delta[k]^(-1)*(-1)^l*sum([chi(x,x,Q)^2*rho_a(x,Q,G_list[i])^(-1)*rho_a(x,Q,G_list[k])^(-1) for x in G_list]) for i in range(n) for j in range(2)] for k in range(n) for l in range(2)])
show(SZZ)
S = block_matrix([[SXX, SXY, SXZ],[SXY.transpose(), SYY, SYZ], [SXZ.transpose(), SYZ.transpose(), SZZ]])/(2*n)
#show(S)
dims = [1]*(2*n) + [2]*ZZ(n*(n-1)/2) + [sqrt_n]*(2*n)
p_plus = sum([Theta[i]*dims[i]^2 for i in range(n*(7+n)/2)])
p_minus = sum([Theta[i]^(-1)*dims[i]^2 for i in range(n*(7+n)/2)])

[          2           2           2           2           2           2           2           2]
[         -2          -2          -2          -2          -2          -2          -2          -2]
[          2           2           2           2           2           2           2           2]
[         -2          -2          -2          -2          -2          -2          -2          -2]
[         -2          -2           2           2  2*zeta16^4  2*zeta16^4  2*zeta16^4  2*zeta16^4]
[          2           2          -2          -2 -2*zeta16^4 -2*zeta16^4 -2*zeta16^4 -2*zeta16^4]
[         -2          -2           2           2  2*zeta16^4  2*zeta16^4  2*zeta16^4  2*zeta16^4]
[          2           2          -2          -2 -2*zeta16^4 -2*zeta16^4 -2*zeta16^4 -2*zeta16^4]

[-2*zeta16^6 + 2*zeta16^2  2*zeta16^6 - 2*zeta16^2  2*zeta16^6 - 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2                        0                        0                        0                        0]
[ 2*zeta16^6 - 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2  2*zeta16^6 - 2*zeta16^2                        0                        0                        0                        0]
[ 2*zeta16^6 - 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2  2*zeta16^6 - 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2                        0                        0                        0                        0]
[-2*zeta16^6 + 2*zeta16^2  2*zeta16^6 - 2*zeta16^2 -2*zeta16^6 + 2*zeta16^2  2*zeta16^6 - 2*zeta16^2                        0                        0                        0                        0]
[                       0                        0                        0                        0          -2*zeta16^4 + 2           2*zeta16^4 - 2          -2*zeta16^4 - 2           2*zeta16^4 + 2]
[                       0                        0                        0                        0           2*zeta16^4 - 2          -2*zeta16^4 + 2           2*zeta16^4 + 2          -2*zeta16^4 - 2]
[                       0                        0                        0                        0          -2*zeta16^4 - 2           2*zeta16^4 + 2          -2*zeta16^4 + 2           2*zeta16^4 - 2]
[                       0                        0                        0                        0           2*zeta16^4 + 2          -2*zeta16^4 - 2           2*zeta16^4 - 2          -2*zeta16^4 + 2]

In [370]:
binomial(n,2)

3

In [444]:
C = S*S
show(C)

22 x 22 dense matrix over Cyclotomic Field of order 16 and degree 8 (use the '.str()' method to see the entries)

In [379]:
ZC_n = 4*n + binomial(n,2)
basis = [vector([0]*i + [1] + [0]*(ZC_n-1-i)) for i in range(ZC_n)]
conj = [basis.index(C*b) for b in basis]
show(conj)
fusions = [matrix([[sum([S[i][r]*S[j][r]*S[conj[k]][r]/S[0][r] for r in range(ZC_n)]) for k in range(ZC_n)] for j in range(ZC_n)]) for i in range(ZC_n)]
for N in fusions:
    show(N)

[0, 1, 4, 5, 2, 3, 7, 6, 8, 9, 10, 13, 14, 11, 12]

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]

[   0    0    1    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    1    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    1    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    1    0    0    0    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0    0    0    0    0    1    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    1    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0  1/2 -1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0 -1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2    0    0]

[   0    0    0    1    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    1    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    1    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    1    0    0    0    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0    0    0    0    0    1    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    1    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0 -1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0  1/2 -1/2]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2    0    0]

...

[   0    0    0    0    0    0    0    0    0    0    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    1    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0  1/2  1/2 -1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0  1/2  1/2  1/2 -1/2]
[   0    0    0    0    0    0    0    0    0    0    0    1    1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    1]
[   0    0    0    0    0    0    0    0    0    1    1    0    0    0    0]
[   0    1    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    1    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    1    0    0    0    0    0    0    0]

[   0    0    0    0    0    0    0    0    0    0    0    1    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0  1/2 -1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0 -1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2    0    0  1/2  1/2]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2    0    0  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    1]
[   0    0    0    0    0    0    0    0    0    1    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    1    0    0]
[   0    0  1/2 -1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    1    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    1    0    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    1    0    0    0    0    0    0]

[   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    0    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0 -1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2  1/2    0    0  1/2 -1/2]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2    0    0  1/2  1/2]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2    0    0  1/2  1/2]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    1]
[   0    0    0    0    0    0    0    0    0    1    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    1    0    0]
[   0    0 -1/2  1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    1    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    1    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    1    0    0    0    0    0    0]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    1]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2  1/2 -1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2 -1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0    1    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    1]
[   0    0  1/2  1/2  1/2 -1/2    0    1    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    1    0    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    1]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    0]
[   0    0    0    0    0    0    0    0    0 -1/2  1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2 -1/2  1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2 -1/2  1/2    0    0]
[   0    0    0    0    0    0    0    0    0  1/2  1/2  1/2 -1/2    0    0]
[   0    0    0    0    0    0    0    0    0    1    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    1]
[   0    0  1/2  1/2 -1/2  1/2    0    1    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    1    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   1    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    1    0    0    0    0    0    0    0    0]

In [374]:
show((S*T^(-1))^3)

[   1    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    1    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2  1/2 -1/2    0    0    0    0    0    0    0    0    0]
[   0    0  1/2  1/2 -1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0  1/2 -1/2  1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0 -1/2  1/2  1/2  1/2    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    1    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    1    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    1    0    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    1    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    1    0    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    1]
[   0    0    0    0    0    0    0    0    0    0    0    1    0    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0]

In [40]:
ind = 42
#Pick one of the TB Category input datas
show(TB_data[ind])
G, met, Q, z, sgn_tau = TB_data[ind]
n = G.order()
n3 = z.multiplicative_order()
sqrt_n = 1
for fac in factor(n.squarefree_part()):
    if fac[0] == 2:
        sqrt_n = z^(n3/8) + z^(7*n3/8)
    else:
        p = fac[0]
        sqrt_n = sqrt_n * (1-z^(n3/4))/(1-z^(n3/4*legendre_symbol(-1,p))) * sum([legendre_symbol(i,p)*z^(n3*i/p) for i in range(p)])
sqrt_n = sqrt_n * sqrt(n/n.squarefree_part())
tau = sgn_tau * sqrt_n/n
#GNN gauss
#gauss = sqrt_n*sum([rho(g, Q) for g in G])/n
#RTJ gauss
gauss = sqrt_n*sum([rho(g, Q)^(-1) for g in G])/n

[
                                                                [8 0]
Additive abelian group isomorphic to Z/3 + Z/3, [(3, 1, 2, 1)], [0 8],

zeta12, 1
]

In [41]:
H = list(Set([2*g for g in G]))
print H
quotient_gens = [g for g in G.gens() if g not in H]
print quotient_gens
coset_reps = [G.zero()] + [sum(K) for K in Set(quotient_gens).subsets()][1:]
print coset_reps
G_list = [c + h for c in coset_reps for h in H]

[(1, 0), (2, 1), (0, 0), (1, 2), (2, 0), (0, 2), (2, 2), (0, 1), (1, 1)]
[]
[(0, 0)]


In [42]:
H = list(Set([2*g for g in G]))
H.sort()
quotient_gens = [g for g in G.gens() if g not in H]
coset_reps = [G.zero()] + [sum(K) for K in Set(quotient_gens).subsets()][1:]
#List elements of G such that they are grouped by cosets of {2g for g in G}
G_list = [c + h for c in coset_reps for h in H]
G_halves = zip(*[[g,2*g] for g in G_list])
c_n = len(coset_reps)
#Building the Deltas with correct +/- is tricky.  First, define them for the coset reps of G / {2g for g in G}
#GNN Deltas
#Delta_for_cos_reps = [(sgn_tau*rho(c, Q)^(-1)*gauss).nth_root(2) for c in coset_reps]
#RTJ Deltas
Delta_for_cos_reps = [(sgn_tau*rho(c, Q)*gauss).nth_root(2)^(-1) for c in coset_reps]
#Then Delta_{2a+c} = Delta_c * chi(-a,a+c) where 2a in H.
Delta = []
for i in range(c_n):
    D = Delta_for_cos_reps[i]
    c = coset_reps[i]
    for h in H:
        #Need to divide h by 2, but Sage does not allow this in Z-modules.
        a = G_halves[0][G_halves[1].index(h)]
        Delta.append(D*chi(a,a+c,Q))
show(Delta)
show(coset_reps)
show(H)
show(G_list)

[-zeta12^3,
 zeta12,
 zeta12,
 zeta12,
 zeta12^3 - zeta12,
 zeta12^3 - zeta12,
 zeta12,
 zeta12^3 - zeta12,
 zeta12^3 - zeta12]

[(0, 0)]

[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]

[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]

In [43]:
def XZ_negation_fix(a,b, Q1, G_list1, Delta1):
    j = G_list1.index(b)
    k = G_list1.index(2*a+b)
    return chi(-a,b+a,Q1)*Delta1[k]/Delta1[j]

correction = []
for a in G_list:
    correction.append([])
    for b in G_list:
        correction[-1:][0].append(XZ_negation_fix(a,b, Q, G_list, Delta))
show(matrix(correction))

[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]

In [44]:
pows = [z^(i) for i in range(n3)]
#Delta = [(tau*sum([rho_a(g,Q,h) for g in G])).nth_root(2)^(-1) for h in G]
Gpairs = [(G_list[i],G_list[j]) for i in range(n) for j in range(i)]
Theta_X = [chi(g,-g,Q) for g in G_list for i in range(2)]
Theta_Y = [chi(g,-h,Q) for (g,h) in Gpairs]
Theta_Z = [sgn_tau*(-1)^i*d^(-1) for d in Delta for i in range(2)]
Theta = Theta_X + Theta_Y + Theta_Z
#show(Theta_X)
#show(Theta_Y)
#show(Theta_Z)
T = block_diagonal_matrix([diagonal_matrix(Theta_X), diagonal_matrix(Theta_Y), diagonal_matrix(Theta_Z)])
#show(T)
SXX = matrix([[chi(g,h,Q)^(2) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#show(SXX)
SXY = matrix([[2*chi(h,a+b, Q)^(1) for (a,b) in Gpairs] for h in G_list for i in range(2)])
#show(SXY)
#SXZ = matrix([[(-1)^(j)/tau*chi(h, g, Q)*XZ_negation_fix(h,g,Q,G_list,Delta) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
SXZ = matrix([[(-1)^(j)/tau*chi(h, g, Q) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
#GNN say SXZ = matrix([[(-1)^(j)*sqrt_n*rho(h,Q)^(-1)*rho_a(h,Q,g)^(-1) for g in G_list for i in range(2)] for h in G_list for j in range(2)])
show(SXZ)
SYY = matrix([[2*(chi(a,d,Q)*chi(b,c,Q)+chi(a,c,Q)*chi(b,d,Q)) for (a,b) in Gpairs] for (c,d) in Gpairs])
#show(SYY)
SYZ = zero_matrix(parent(z), binomial(n,2), 2*n)
#show(SYZ)
SZZ = matrix([[Delta[i]^(-1)*(-1)^j*Delta[k]^(-1)*(-1)^l*sum([chi(x,x+G_list[i]+G_list[k],Q)^(-1) for x in G_list]) for i in range(n) for j in range(2)] for k in range(n) for l in range(2)])
show(SZZ)
S = block_matrix([[SXX, SXY, SXZ],[SXY.transpose(), SYY, SYZ], [SXZ.transpose(), SYZ.transpose(), SZZ]])/(2*n)
#show(S)
dims = [1]*(2*n) + [2]*ZZ(n*(n-1)/2) + [sqrt_n]*(2*n)
p_plus = sum([Theta[i]*dims[i]^2 for i in range(n*(7+n)/2)])
p_minus = sum([Theta[i]^(-1)*dims[i]^2 for i in range(n*(7+n)/2)])

[              3               3               3               3               3               3               3               3               3               3               3               3               3               3               3               3               3               3]
[             -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3              -3]
[              3               3  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2               3               3  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2               3               3  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2]
[             -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2              -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2              -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2]
[              3               3     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3               3               3     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3               3               3     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3]
[             -3              -3      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3              -3              -3      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3              -3              -3      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3]
[              3               3               3               3               3               3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2]
[             -3              -3              -3              -3              -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2]
[              3               3  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3     -3*zeta12^2     -3*zeta12^2               3               3     -3*zeta12^2     -3*zeta12^2               3               3  3*zeta12^2 - 3  3*zeta12^2 - 3]
[             -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3      3*zeta12^2      3*zeta12^2              -3              -3      3*zeta12^2      3*zeta12^2              -3              -3 -3*zeta12^2 + 3 -3*zeta12^2 + 3]
[              3               3     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3               3               3     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3               3               3]
[             -3              -3      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*zeta12^2 + 3              -3              -3      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3              -3              -3]
[              3               3               3               3               3               3     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2     -3*zeta12^2  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3  3*zeta12^2 - 3]
[             -3              -3              -3              -3              -3              -3      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2      3*zeta12^2 -3*zeta12^2 + 3 -3*zeta12^2 + 3 -3*ze

[              3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3]
[             -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3              -3               3]
[              3              -3     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3               3              -3     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3               3              -3     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3]
[             -3               3      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3              -3               3      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3              -3               3      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3]
[              3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2               3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2               3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2]
[             -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2              -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2              -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2]
[              3              -3               3              -3               3              -3     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3]
[             -3               3              -3               3              -3               3      3*zeta12^2     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3]
[              3              -3     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2  3*zeta12^2 - 3 -3*zeta12^2 + 3               3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3               3              -3     -3*zeta12^2      3*zeta12^2]
[             -3               3      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2 -3*zeta12^2 + 3  3*zeta12^2 - 3              -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3              -3               3      3*zeta12^2     -3*zeta12^2]
[              3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2               3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2               3              -3]
[             -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2      3*zeta12^2     -3*zeta12^2              -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2              -3               3]
[              3              -3               3              -3               3              -3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2     -3*zeta12^2      3*zeta12^2]
[             -3               3              -3               3              -3               3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3 -3*zeta12^2 + 3  3*zeta12^2 - 3      3*zeta12^2     -3*zeta12^2      

In [45]:
C = S^2
show(C)

72 x 72 dense matrix over Cyclotomic Field of order 12 and degree 4 (use the '.str()' method to see the entries)

In [537]:
ZC_n = 4*n + binomial(n,2)
basis = [vector([0]*i + [1] + [0]*(ZC_n-1-i)) for i in range(ZC_n)]
conj = [basis.index(C*b) for b in basis]
show(conj)
fusions = [matrix([[sum([S[i][r]*S[j][r]*S[conj[k]][r]/S[0][r] for r in range(ZC_n)]) for j in range(ZC_n)] for k in range(ZC_n)]) for i in range(ZC_n)]
for N in fusions:
    show(N)

[0, 1, 2, 3, 4, 5, 6, 7, 8]

[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]

[0 1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]

[0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]

[0 0 0 1 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]

[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[1 1 1 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1]
[0 0 0 0 0 0 0 1 1]
[0 0 0 0 0 1 1 0 0]
[0 0 0 0 0 1 1 0 0]

[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 1 1]
[1 0 0 1 0 0 0 0 0]
[0 1 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]

[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 1]
[0 1 1 0 0 0 0 0 0]
[1 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]

[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 1 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[1 0 1 0 0 0 0 0 0]
[0 1 0 1 0 0 0 0 0]

[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 1 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 1 0 1 0 0 0 0 0]
[1 0 1 0 0 0 0 0 0]

In [46]:
show((S*T)^3)

72 x 72 dense matrix over Cyclotomic Field of order 12 and degree 4 (use the '.str()' method to see the entries)

In [47]:
(S*T)^3 == S^2

True

In [48]:
latex(S)

\left(\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr}
\frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{18} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{9} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{

In [49]:
latex(T)

\left(\begin{array}{rrrrrrrrrrrrrrrrrr|rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr|rrrrrrrrrrrrrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & -\zeta_{12}^{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & -\zeta_{12}

In [50]:
show(T)

72 x 72 sparse matrix over Cyclotomic Field of order 12 and degree 4 (use the '.str()' method to see the entries)

In [25]:
show(G_list)

[(1, 0), (2, 1), (0, 0), (1, 2), (2, 0), (0, 2), (2, 2), (0, 1), (1, 1)]

In [27]:
H.sort()
show(H)

[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]

In [9]:
met_groups

[[(2, 1, 1, 1)],
 [(3, 1, 1, 1)],
 [(3, 1, 1, 2)],
 [(2, 2, 1, 1)],
 [(2, 2, 1, 2)],
 [(2, 1, 2, 1)],
 [(2, 1, 2, 5)],
 [(5, 1, 1, 1)],
 [(5, 1, 1, 2)],
 [(2, 1, 1, 1), (3, 1, 1, 1)],
 [(2, 1, 1, 1), (3, 1, 1, 2)],
 [(7, 1, 1, 1)],
 [(7, 1, 1, 2)],
 [(2, 3, 1, 1)],
 [(2, 3, 1, 2)],
 [(2, 3, 1, 3)],
 [(2, 3, 1, 4)],
 [(2, 1, 1, 1), (2, 2, 1, 1)],
 [(2, 1, 3, 1)],
 [(3, 2, 1, 1)],
 [(3, 2, 1, 2)],
 [(3, 1, 2, 1)],
 [(2, 1, 1, 1), (5, 1, 1, 1)],
 [(2, 1, 1, 1), (5, 1, 1, 2)],
 [(11, 1, 1, 1)],
 [(11, 1, 1, 2)],
 [(2, 2, 1, 1), (3, 1, 1, 1)],
 [(2, 2, 1, 2), (3, 1, 1, 1)],
 [(2, 2, 1, 1), (3, 1, 1, 2)],
 [(2, 2, 1, 2), (3, 1, 1, 2)],
 [(2, 1, 2, 1), (3, 1, 1, 1)],
 [(2, 1, 2, 5), (3, 1, 1, 1)],
 [(2, 1, 2, 1), (3, 1, 1, 2)],
 [(2, 1, 2, 5), (3, 1, 1, 2)],
 [(13, 1, 1, 1)],
 [(13, 1, 1, 2)],
 [(2, 1, 1, 1), (7, 1, 1, 1)],
 [(2, 1, 1, 1), (7, 1, 1, 2)]]

In [37]:
i = 7
a = list(Bilinear_Forms.prd_tuples)[i]

In [38]:
list(a)

[(5, 1, 1)]